In [137]:
import pandas as pd
import nbimporter
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from Helper import mapping as mapped_values
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [138]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

In [139]:
def read_train(path):
    df_training = pd.read_csv(path, index_col=0)
    return df_training

def read_test(path):
    df_testing = pd.read_csv(path, index_col= 0)
    return df_testing

In [140]:
def create_predictors(df_train, df_test=None):
    df_train['date'] = pd.to_datetime(df_train['date'])

    df_train['venue_code'] = df_train['venue'].astype('category').cat.codes
    df_train['opp_code'] = df_train['opponent'].astype('category').cat.codes

    df_train['hour'] = df_train['time'].str.replace(":.+", "",regex=True)
    df_train['hour'] = df_train['hour'].fillna(df_train['hour'].value_counts().idxmax())

    df_train['day_code'] = df_train['date'].dt.dayofweek
    df_train['day_code'] = df_train['day_code'].fillna(df_train['day_code'].value_counts().idxmax())

    # df_train['target'] = (df_train['result'] == 'W').astype('int')
    result_mapping = {'L': 0, 'W': 1, 'D': 2}

    df_train['target'] = df_train['result'].map(result_mapping)

    if df_test is not None: # Đổi '!= None' thành 'is not None' cho Pythonic hơn
        df_test['date'] = pd.to_datetime(df_test['date'])

        df_test['venue_code'] = df_test['venue'].astype('category').cat.codes

        df_test['opp_code'] = df_test['opponent'].astype('category').cat.codes

        df_test['hour'] = df_test['time'].str.replace(":.+", "",regex=True)
        df_test['hour'] = df_test['hour'].fillna(df_test['hour'].value_counts().idxmax())

        df_test['day_code'] = df_test['date'].dt.dayofweek

        # df_test['target'] = (df_test['result'] == 'W').astype('int')
        df_test['target'] = df_test['result'].map(result_mapping)

        return df_train, df_test

    return df_train

In [141]:
# Calculate the rolling average for the last 5 games
# and replace NaN values with the rolling average.
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(5, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

def update_with_rolling_average(df, cols, new_cols):
    matches_rolling = df.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
    matches_rolling = matches_rolling.droplevel('team')
    matches_rolling.index = range(matches_rolling.shape[0])
    return matches_rolling

In [142]:
url_ita = "../Datasets/Cleaned Datasets/Serie_A_Stats_Italy.csv"
url_spain = "../Datasets/Cleaned Datasets/La_Liga_Stats.csv"
urls = [url_ita, url_spain]
def combine(combined, matches_rolling):
    combined_data = combined.merge(matches_rolling[['date', 'time', 'team', 'opponent', 'result']],
                                   left_index = True, right_index = True)
    return combined_data

def train_model(data, predictors, url):
    rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=42)

    if url == url_spain:
        train = data[data["date"] <= '2023-07-19']
        test = data[data["date"] > '2023-07-23']
    else:
        train = data[data["date"] <= '2025-01-02']
        test = data[data["date"] > '2025-01-02']

    print(f"Train set size for {url}: {len(train)}")
    print(f"Test set size for {url}: {len(test)}")

    if len(train) == 0 or len(test) == 0:
        print(f"[ERROR] One of the datasets is empty for: {url}")
        return None, None

    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)

    return rf, combined


In [143]:
def map_values(combined_df):

    mapping_values = mapped_values() # Gọi hàm mapped_values từ Helper.ipynb

    for map_values_dict in mapping_values: # Đổi tên biến để tránh nhầm lẫn với hàm
        mapping = MissingDict(**map_values_dict)

        combined_df["new_team"] = combined_df["team"].map(mapping)

        # Merge DataFrame với chính nó để tạo cặp đấu
        merged = combined_df.merge(combined_df, left_on=["date", 'opponent'], right_on=["date", 'new_team'], suffixes=('_x', '_y'))
        # Đảm bảo các cột được chọn đúng
        merged = merged[['date', 'time_x', 'new_team_x', 'new_team_y', 'predicted_x', 'predicted_y']]

        map_string = {0: 'L', 1 : 'W', 2 : 'D'}

        merged['new_predicted_x'] = merged['predicted_x'].map(map_string)
        merged['new_predicted_y'] = merged['predicted_y'].map(map_string)

        renamed_columns = {
            'date' : 'Date',
            'time_x' : 'Time',
            'new_team_x' : 'Team A',
            'new_team_y' : 'Team B',
            'new_predicted_x' : 'Prediction for Team A',
            'new_predicted_y' : 'Prediction for Team B'
        }

        merged.rename(columns=renamed_columns, inplace=True)

        merged = merged.drop(['predicted_x', 'predicted_y'], axis = 1)

    return merged

In [144]:
def drop_teams(df, url):
    # Danh sách các đội cần loại bỏ cho giải Ý
    teams_to_drop_ita = [
        "at Rapid Wien",
        "Reggiana", "Parma",
        "FeralpiSalò", "Spezia",
        "Cosenza", "Sampdoria",
        "Ternana", "Pisa", "Como",
        "Ascoli", "Modena", "Cittadella",
        "Palermo", "Cesena", "Catanzaro"
    ]

    # Danh sách các đội cần loại bỏ cho giải Tây Ban Nha
    teams_to_drop_spain = [
        "Manchester Utd", "Bayern Munich", "Juventus", "Manchester City", "Chelsea", "Paris S-G", "Liverpool",
        "Leverkusen", "Dortmund", "Roma", "Napoli", "Shakhtar", "Lyon", "PSV Eindhoven", "Milan", "Ajax", "Celtic",
        "Inter", "Porto", "Zenit", "RB Salzburg", "Krasnodar", "Sevilla", "Sporting CP", "Galatasaray", "Arsenal",
        "M'Gladbach", "Viktoria Plzeň", "Benfica", "Club Brugge", "Valencia", "Monaco", "Olympiacos", "Panathinaikos",
        "Loko Moscow", "APOEL FC", "Ludogorets", "Spartak Moscow", "Rapid Wien", "Dinamo Zagreb", "BATE Borisov",
        "Rennes", "Dynamo Kyiv", "Basel", "Leonesa", "Alcoyano", "FC Astana", "CD Mirandés", "Schalke 04", "Rijeka",
        "FC Copenhagen", "AZ Alkmaar", "Ferencváros", "Genk", "Mirandés", "Rubin Kazan", "SD Formentera", "Lazio",
        "Standard Liège", "Slavia Prague", "Hannover 96", "Sparta Prague", "Atalanta", "Lille", "CSKA Moscow", "Lleida",
        "Athletic Club", "RB Leipzig", "Ponferradina", "Eint Frankfurt", "Alcorcón", "Tenerife", "Malmö", "Marseille",
        "Barcelona", "Fuenlabrada", "Rangers", "Young Boys", "Ebro", "Leicester City", "Cornellà", "Numancia", "Wolves",
        "Albacete", "UD Ibiza", "Trabzonspor", "CP Cacereño", "Real Murcia", "CD Arenteiro", "Oviedo", "AD Ceuta",
        "Atlético Baleares", "Gimnàstic", "Zamora CF", "Académica", "Žalgiris", "Sigma Olomouc", "Motherwell",
        "Akhisarspor", "Jablonec", "Hapoel Tel Aviv", "Union Berlin", "St. Gallen", "Kuban", "Swansea City", "Lugo",
        "Újpest", "Twente", "Helsingborg", "Melilla", "Betis", "Braga", "Sant Andreu", "Slaven Belupo", "Kiryat Shmona",
        "Antwerp", "Stoke City", "Strømsgodset", "Udinese", "Beşiktaş", "Luzern", "ŠK Slovan Bratislava", "Stjarnan",
        "Qarabağ FK", "Steaua", "Slovan Liberec", "Reus", "MŠK Žilina", "Inter Baku", "Dinamo Minsk", "Dinamo",
        "Hertha BSC", "Östersund", "Real Jaén", "Partizan", "Legia Warsaw", "Hércules", "Rostov", "Sassuolo", "Guijuelo",
        "Toledo", "Augsburg", "FK Vardar", "Osmanlıspor", "Apollon Limassol", "Estoril", "Śląsk Wrocław", "Mladost",
        "Aberdeen", "Torino", "Başakşehir", "Fiorentina", "Linense", "Sabadell", "L'Hospitalet", "Llagostera",
        "Rosenborg", "Barakaldo", "Gent", "Freiburg", "Odense", "Atlético Mancha Real", "Intercity", "CD Rincón", "APOEL",
        "UD Llanera", "CFJ Mollerussa", "Aris Limassol FC", "Marbella", "Maccabi Haifa", "Real Unión", "Fenerbahçe",
        "Club Portugalete", "Lens", "CF La Nucía", "Navalcarnero", "Sivasspor", "CFR Cluj", "Pontevedra", "West Ham",
        "Bergantiños FC", "Sturm Graz", "Hajduk Split", "Lech Poznań", "Be'er Sheva", "CD Guijuelo", "FC Andorra",
        "SD Leioa", "Badajoz", "Badalona", "Unionistas Sal", "Sestao", "PAOK", "Andratx", "CF Independiente Alicante",
        "Majadahonda", "Guadalajara", "CD L'Alcora", "CF Talavera de la Reina", "Club Deportivo Atlético Paso",
        "CF Panadería Pulido", "Recreativo", "Arenas Club de Getxo", "CD Autol", "CD Cazalegas", "CD Coria",
        "CD Santa Amalia", "Ibiza", "CD Fuentes", "CD Arnedo", "UD Alzira", "CD Eldense", "Quintanar del Rey",
        "UD Barbadás", "Atlético Saguntino", "Velarde CF", "Juventud Torremolinos CF", "Victoria CF", "Gernika Club",
        "CD San Roque de Lepe", "CD Diocesano", "Dnipro", "CD Algar", "Solares SD", "Atlético Sanluqueño CF",
        "CE L'Hospitalet", "CD Cantolagua", "Peña Deportiva", "Racing Rioja CF", "Las Rozas CF", "UM Escobedo",
        "Comillas", "Orihuela CF", "Becerril", "UD Sanse"
    ]

    # Lấy tên file từ URL để xác định giải đấu
    filename = url.split('/')[-1]

    # Kiểm tra tên file và loại bỏ các đội tương ứng
    if filename == 'Serie_A_Stats_Italy.csv':
        df = df[~df['opponent'].isin(teams_to_drop_ita)]
    elif filename == 'La_Liga_Stats.csv': # Đã đổi từ Primera_Division_Stats.csv sang La_Liga_Stats.csv
        df = df[~df['opponent'].isin(teams_to_drop_spain)]

    return df

In [145]:
def export_model(model, name):
    """
    Hàm này dùng để xuất các model đã được huấn luyện với các tập dữ liệu khác nhau.
    Các tham số sử dụng:
        - model: model đã được huấn luyện
        - name: một biến để xác định từng model. Nó được sử dụng trong hàm main,
                và có thể được gán trong mỗi câu lệnh if với tên tương ứng.
                Ví dụ: name = "brazil_serie_a"
    """
    export = pickle.dump(model, open('../rf_' + name +'.pkl', 'wb'))
    return export

In [146]:
if __name__ == "__main__":

    url_ita = "../Datasets/Cleaned Datasets/Serie_A_Stats_Italy.csv"
    url_spain = "../Datasets/Cleaned Datasets/La_Liga_Stats.csv"


    urls = [url_ita, url_spain]
    
    for idx, url in enumerate(urls):
        
        df_train = read_train(url)
        
        if  url == url_ita or url == url_spain:
            df_train = drop_teams(df_train, url)

        df_train = create_predictors(df_train)
                
        cols = ["gf", "ga", "sh", "sot", "pk", "pkatt"]
        new_cols = [f"{c}_rolling" for c in cols]
        matches_rolling = update_with_rolling_average(df_train, cols, new_cols)
        
        predictors = ['venue_code', 'opp_code', 'hour', 'day_code']
        rf_model, combined = train_model(matches_rolling, predictors + new_cols, url)

        combined_df = combine(combined, matches_rolling)
        # combined_df = combined_df.drop(['actual', 'result'], axis = 1)
    
        final_df = map_values(combined_df)

        # Export the DataFrame to a CSV file

        # Xuất DataFrame ra file CSV và model Pickle
        if idx == 0: # Giải Ý
            final_df.to_csv('../Datasets/Predictions/predictions_serieA_italy.csv', index=False)
            export_model(rf_model, "serieA_italy")
        elif idx == 1: # Giải Tây Ban Nha
            final_df.to_csv('../Datasets/Predictions/predictions_laliga_spain.csv', index=False)
            export_model(rf_model, "laliga_spain")

Train set size for ../Datasets/Cleaned Datasets/Serie_A_Stats_Italy.csv: 8179
Test set size for ../Datasets/Cleaned Datasets/Serie_A_Stats_Italy.csv: 427
Train set size for ../Datasets/Cleaned Datasets/La_Liga_Stats.csv: 8086
Test set size for ../Datasets/Cleaned Datasets/La_Liga_Stats.csv: 615
